# Testing the implementation

In [ ]:
import os
import gzip
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

from models import *
from trainer import *
from preprocess import *
from callbacks import *

## 0 Data

In [ ]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/processed_data.csv'))
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'optionid', 'securityid'], axis=1)
df.sample(5)

Create the first neueral network that uses the same set of inputs as the *Black-Scholes model*

In [ ]:
dataframe_BS = np.vstack((df['strike'].values,
                      df['asset_price'].values,
                      df['days_to_maturity'].values,
                      df['volatility'].values,
                      df['rate'].values,
                      df['contract_price'].values)).T                

In [ ]:
train_ds, valid_ds, test_ds = pipeline1(dataframe_BS, scaling=False)

train_copy, valid_copy, test_copy = pipeline1(dataframe_BS, prefetch=False)

## 1 Start training

Define some parameters

In [ ]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/')  # path to save the model

patience = 10  

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss = tf.keras.losses.MeanAbsoluteError(name='loss')

metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='accuracy')

num_epochs = 10  

input_shape = (5,)

num_layers = 3

hidden_units = [14, 14, 14]

output_shape = (1, )

batchnorm = True  

dropout = None

model = getModel(input_shape = input_shape,
                num_layers   = num_layers,
                 hidden_units = hidden_units,
                 output_shape = output_shape,
                 batchnorm = batchnorm,
                 dropout = dropout)
model.summary()

Define some callbacks  

In [ ]:
ckpt = CheckpointCallback(path_to_save)
printing =PrintProgress(num_epochs=print_num_epochs)
early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')
callbacks = [ckpt, printing, early_stop]

In [ ]:
# history = compile_and_fit(model,
#                           optimizer,
#                           loss,
#                           num_epochs,
#                           train_ds,
#                           valid_ds,
#                           metrics,
#                           callbacks
#                           )

# 1.1 Tuning the hyperparameters

To tune the hyperparameter, we can use `keras_tuner`, which provides `RandomSearch`, `Hyperband`, and `BayesianSearch` methods for searching the best hyperparameters.  

To get started, we only tune the number of layers and the hidden units in each layer first, then we tune with learning rate together.

In [ ]:
import keras_tuner

We initialize the `keras_tuner` object for tuning hyperparameters; currently, there are two hyper models implemented, `tuneLayer` and `tuneLR`

In [25]:
random_tuner = keras_tuner.RandomSearch(
    hypermodel=tuneLR, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", ## the objective to optimize
    max_trials=3, # the maximum number of trials to run
    executions_per_trial=2, # the number of models generated on each trial
    overwrite=True, # whether to overwrite previous trials
    directory="hyperparams/RandomSearch", # the directory to save the trials
    project_name="1", # the name of the project
)  

# get a summary of the range of hyperparameters to tune
random_tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 50, 'step': 5, 'sampling': None}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
# start the search
random_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)    

Get the best compiled models

In [ ]:
models = random_tuner.get_best_models(num_models=1)
best_model = models[0]

best_model.summary()

We can view the results of the search and use the hyperparameters displayed to train the model.

In [ ]:
random_tuner.results_summary()

We also try `Hyperband` which generates a large number of models and discard them in a tournament style.

In [ ]:
band_tuner = keras_tuner.Hyperband(
    hypermodel=tuneLR,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparams/Hyperband",
    project_name="1"
)
band_tuner.search_space_summary()
band_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)  

In [ ]:
band_tuner.results_summary()

Finally, we try `BayesianOptimization`.

In [27]:
bayes_tuner = keras_tuner.BayesianOptimization(
    hypermodel=tuneLR,
    objective="val_loss",
    max_trials=3,
    directory="hyperparams/BayesianOptimization",
    project_name="1"
)
bayes_tuner.search_space_summary()
bayes_tuner.search(train_ds, epochs = 2, validation_data = valid_ds) 

Trial 3 Complete [00h 00m 01s]
val_loss: 2.6899940967559814

Best val_loss So Far: 2.6899940967559814
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit


In [28]:
bayes_tuner.results_summary()

Results summary
Results in hyperparams/BayesianOptimization/1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 1
units_1: 5
learning_rate: 0.01
units_2: 50
units_3: 50
units_4: 50
units_5: 50
Score: 2.6899940967559814
Trial summary
Hyperparameters:
num_layers: 4
units_1: 20
learning_rate: 0.000881825746836976
units_2: 35
units_3: 25
units_4: 45
units_5: 35
Score: 3.1345088481903076
Trial summary
Hyperparameters:
num_layers: 5
units_1: 30
learning_rate: 0.00010014380923497382
units_2: 5
units_3: 5
units_4: 5
units_5: 5
Score: 4.473343372344971
